# PostgreSQL

## SELECT Query

```sql
SELECT DISTINCT column1, column2, NOW() AS current_time
FROM table_name
WHERE condition1
    AND condition2
    OR condition3
ORDER BY column1 ASC, column2 DESC NULLS LAST -- can also be NULLS FIRST
OFFSET 10 LIMIT 10;
```

## Operators

| Operator | Description |
| --- | --- |
| `=`, `<>`, `!=`, `>`, `<`, `>=`, `<=` | Equal, Not Equal, Greater Than, Less Than, Greater Than or Equal, Less Than or Equal |
| `::` | Cast a value to a different data type |
| `BETWEEN` | Between a range, like `payment_date BETWEEN '2007-02-15' AND '2007-02-20'` |
| `IN` | In a list of values, like `col_name IN ('val1', 'val2')` |
| `LIKE` | Pattern matching, `%` for any number of characters, `_` for a single character |
| `IS NULL`, `IS NOT NULL` | Check if value is NULL or not NULL |
| `AND`, `OR`, `NOT` | Logical operators |
| `ANY`, `ALL` | Compare a value to a set of values, like `col_name > ANY (1, 2, 3)` |
| `EXISTS`, `NOT EXISTS` | Check if a subquery returns any rows |

## JOINs

`INNER JOIN`, `LEFT JOIN` = `LEFT OUTER JOIN`, `RIGHT JOIN` = `RIGHT OUTER JOIN`, `FULL JOIN` = `FULL OUTER JOIN`, `CROSS JOIN`, `NATURAL JOIN` are supported.

## UNION, UNION ALL, INTERSECT, EXCEPT

`UNION` removes duplicates, `UNION ALL` keeps duplicates, `INTERSECT` returns common rows, `EXCEPT` returns rows in first query but not in second query.

## Boolean Operators

- Boolean can have three values: `true`, `false`, and `null`.
- `true` -> true, `t`, `y`, `yes`, `true`, `1`
- `false` -> false, `f`, `n`, `no`, `false`, `0`

## GROUP BY (grouping sets)

```sql
SELECT
	GROUPING(brand) grouping_brand, -- 1 if brand is NULL, 0 otherwise
	GROUPING(segment) grouping_segment,
	brand,
	segment,
	SUM (quantity)
FROM
	sales
GROUP BY
	GROUPING SETS ( -- multiple GROUP BYs, 
        (brand, segment), -- group by brand and segment
		(brand), -- group by brand, segment is NULL
		(segment), -- group by segment, brand is NULL
		() -- no grouping
	)
HAVING GROUPING(brand) = 0 -- only show rows where brand is not NULL
ORDER BY
	brand,
	segment;
```

## GROUP BY (full rollup)

```sql
SELECT segment, brand, SUM (quantity) FROM sales 
GROUP BY 
    ROLLUP (segment, brand) -- full rollup, generates all levels in hierarchy
ORDER BY segment, brand
-- above is equivalent to
SELECT segment, brand, SUM (quantity) FROM sales 
GROUP BY 
    GROUPING SETS (
      (segment, brand),
      (segment),
      ()
    )
ORDER BY segment, brand
```

## GROUP BY (partial rollup)

```sql
SELECT segment, brand, SUM (quantity) FROM sales 
GROUP BY 
    segment, ROLLUP (brand) -- partial rollup
ORDER BY segment, brand
-- above is equivalent to
SELECT segment, brand, SUM (quantity) FROM sales 
GROUP BY 
    GROUPING SETS (
      (segment, brand),
      (segment)
    )
ORDER BY segment, brand
```

## GROUP BY (cube)

```sql
SELECT segment, brand, SUM (quantity) FROM sales
GROUP BY
    CUBE (brand, segment) -- generate all possible combinations
ORDER BY segment, brand
-- above is equivalent to
SELECT segment, brand, SUM (quantity) FROM sales
GROUP BY
    GROUPING SETS (
      (segment, brand),
      (segment),
      (brand),
      ()
    )
ORDER BY segment, brand
```

## ANY and ALL

```sql
SELECT - FROM employees
WHERE salary < ALL (SELECT salary FROM managers)
-- use ANY / ALL with <, >, =, <=, >=, <> and subquery
ORDER BY salary DESC;
```

## EXISTS

```sql
SELECT first_name, last_name FROM customer c
WHERE EXISTS ( -- check if subquery returns any rows
    SELECT 1 FROM payment p
    WHERE p.customer_id = c.customer_id AND amount > 11 -- correlated subquery
)
```

## Common Table Expressions (CTE)

```sql
WITH film_stats AS (
    SELECT AVG(rental_rate) AS avg_rental_rate, MAX(length) AS max_length, MIN(length) AS min_length
    FROM film
), customer_stats AS (
    SELECT COUNT(DISTINCT customer_id) AS total_customers, SUM(amount) AS total_payments
    FROM payment
), main_query AS (
    SELECT
        ROUND((SELECT avg_rental_rate FROM film_stats), 2) AS avg_film_rental_rate,
        (SELECT max_length FROM film_stats) AS max_film_length,
        (SELECT min_length FROM film_stats) AS min_film_length,
        (SELECT total_customers FROM customer_stats) AS total_customers,
        (SELECT total_payments FROM customer_stats) AS total_payments
) SELECT - FROM main_query;
```

## Recursive CTE

- use `WITH RECURSIVE` to create recursive CTE
```sql
WITH RECURSIVE subordinates AS (
  SELECT employee_id, manager_id, full_name FROM employees
  WHERE employee_id = 2
  UNION
  SELECT e.employee_id, e.manager_id, e.full_name FROM employees e
    INNER JOIN subordinates s ON s.employee_id = e.manager_id
)
SELECT - FROM subordinates;
```

## Modifying Data, inserting rows

```sql
CREATE TABLE contacts (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(384) NOT NULL UNIQUE
);
```

```sql
INSERT INTO contacts (first_name, last_name, email)
VALUES
    ('John', 'Doe', '[[email protected]](../cdn-cgi/l/email-protection1.html)'),
    ('Jane', 'Smith', '[[email protected]](../cdn-cgi/l/email-protection2.html)'),
    ('Bob', 'Johnson', '[[email protected]](../cdn-cgi/l/email-protection3.html)')
RETURNING *;
```

## Modifying Data, updating rows

```sql
CREATE TABLE courses(
  course_id serial PRIMARY KEY,
  course_name VARCHAR(255) NOT NULL,
  price DECIMAL(10,2) NOT NULL,
  description VARCHAR(500),
  published_date date
);

INSERT INTO courses( course_name, price, description, published_date)
VALUES
('PostgreSQL for Developers', 299.99, 'A complete PostgreSQL for Developers', '2020-07-13'),
('PostgreSQL Admininstration', 349.99, 'A PostgreSQL Guide for DBA', NULL),
('PostgreSQL High Performance', 549.99, NULL, NULL),
('PostgreSQL Bootcamp', 777.99, 'Learn PostgreSQL via Bootcamp', '2013-07-11'),
('Mastering PostgreSQL', 999.98, 'Mastering PostgreSQL in 21 Days', '2012-06-30');

UPDATE courses
SET published_date = '2020-08-01'
WHERE course_id = 3;

UPDATE courses
SET price = price - 1.05;
```

## Modifying Data, upserting rows

```sql
INSERT INTO inventory (id, name, price, quantity)
VALUES (1, 'A', 16.99, 120)
ON CONFLICT(id)
DO UPDATE SET
  price = EXCLUDED.price,
  quantity = EXCLUDED.quantity;
```

## Transactions

- Operations that maintain data integrity using BEGIN, COMMIT, and ROLLBACK statements to ensure all-or-nothing execution
* Ensures data consistency through atomic operations using BEGIN, COMMIT, and ROLLBACK
* Supports isolation levels: READ COMMITTED, REPEATABLE READ, SERIALIZABLE
* Can create SAVEPOINTs for partial rollbacks
```sql
BEGIN;
SAVEPOINT before_update;
UPDATE accounts SET balance = balance - 100 WHERE id = 1;
-- Something went wrong? Roll back to savepoint
ROLLBACK TO before_update;
-- Or commit if all is well
COMMIT;
```

## Data Import/Export
* COPY command supports both import and export operations
* Handles CSV, TEXT, and BINARY formats
* Supports column selection and data transformations
```sql
-- Export with specific columns and formatting
COPY (
    SELECT name, salary, hire_date::text 
    FROM employees 
    WHERE department = 'Sales'
) TO '/path/to/sales_team.csv' 
WITH (FORMAT CSV, HEADER, DELIMITER ',', QUOTE '"');

-- Import with error handling
COPY employees 
FROM '/path/to/new_employees.csv' 
WITH (
    FORMAT CSV, 
    HEADER, 
    DELIMITER ',',
    FORCE_NULL (manager_id, bonus)
);
```

## Table Management
* Create tables with various options and constraints
* Modify existing tables using ALTER TABLE
* Support for temporary and unlogged tables
* Table partitioning for better performance
```sql
-- Create a partitioned table
CREATE TABLE sales (
    sale_id SERIAL,
    sale_date DATE NOT NULL,
    amount NUMERIC(10,2),
    customer_id INT
) PARTITION BY RANGE (sale_date);

-- Create partitions
CREATE TABLE sales_2024_q1 PARTITION OF sales
    FOR VALUES FROM ('2024-01-01') TO ('2024-04-01');

-- Add/modify columns
ALTER TABLE sales 
    ADD COLUMN region TEXT,
    ALTER COLUMN amount SET NOT NULL,
    DROP COLUMN IF EXISTS old_field;

-- Create a table from query results
SELECT customer_id, SUM(amount) as total_spent
INTO TEMPORARY TABLE high_value_customers
FROM sales
WHERE amount > 1000
GROUP BY customer_id;
```

## Constraints
* Primary Key: Unique identifier for rows
* Foreign Key: Maintains referential integrity
* Check: Custom validation rules
* Unique: Prevents duplicate values
* Exclusion: Complex row-level constraints
```sql
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    sku VARCHAR(50) UNIQUE NOT NULL,
    name TEXT NOT NULL,
    price NUMERIC(10,2) CHECK (price > 0),
    weight NUMERIC(5,2) CHECK (weight > 0),
    dimensions JSONB,
    CONSTRAINT valid_dimensions CHECK (
        (dimensions->>'length')::numeric > 0 AND
        (dimensions->>'width')::numeric > 0 AND
        (dimensions->>'height')::numeric > 0
    )
);

CREATE TABLE inventory (
    product_id INT REFERENCES products ON DELETE CASCADE,
    warehouse_id INT REFERENCES warehouses,
    quantity INT DEFAULT 0,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (product_id, warehouse_id)
);
```

## Data Types
* Numeric: INTEGER, BIGINT, NUMERIC, REAL, DOUBLE PRECISION
* Character: CHAR, VARCHAR, TEXT
* Temporal: DATE, TIME, TIMESTAMP, INTERVAL
* Binary: BYTEA
* Arrays: Any type can be an array
* JSON/JSONB: Store and query JSON data
* Geometric: POINT, LINE, CIRCLE
* Custom: ENUM, Composite Types
```sql
CREATE TYPE user_status AS ENUM ('active', 'inactive', 'banned');
CREATE TYPE address AS (
    street TEXT,
    city TEXT,
    postal_code VARCHAR(10)
);

CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    email VARCHAR(255) UNIQUE NOT NULL,
    status user_status DEFAULT 'active',
    settings JSONB DEFAULT '{}',
    login_history TIMESTAMP[],
    shipping_address address,
    location POINT,
    profile_image BYTEA,
    last_login TIMESTAMP WITH TIME ZONE,
    session_duration INTERVAL
);
```

## Conditional Expressions and Operators
* CASE: Simple and searched CASE expressions
* COALESCE: Returns first non-null value
* NULLIF: Returns null if values are equal
* GREATEST/LEAST: Returns highest/lowest value
* Type casting using CAST or :: operator
```sql
SELECT 
    product_name,
    price,
    inventory_count,
    CASE 
        WHEN inventory_count = 0 THEN 'Out of Stock'
        WHEN inventory_count < 10 THEN 'Low Stock'
        WHEN inventory_count < 50 THEN 'Medium Stock'
        ELSE 'Well Stocked'
    END as stock_status,
    COALESCE(discount, 0) as effective_discount,
    NULLIF(returns, 0) as return_count,
    GREATEST(regular_price, sale_price) as display_price,
    price::text || ' USD' as formatted_price
FROM inventory_status;
```

## Utilities and Administration
* psql: Command-line client
* EXPLAIN: Query performance analysis
* VACUUM: Database maintenance
* Roles and permissions management
```sql
-- Analyze query performance
EXPLAIN (ANALYZE, BUFFERS, FORMAT JSON)
SELECT * FROM orders 
WHERE created_at >= CURRENT_DATE - INTERVAL '30 days'
AND status = 'completed';

-- Create role with specific permissions
CREATE ROLE reporting_user 
WITH LOGIN PASSWORD 'secure123'
VALID UNTIL '2025-12-31';

GRANT SELECT ON ALL TABLES IN SCHEMA public TO reporting_user;
```

## Aggregate Functions

| Function | Description | Example | Result |
|----------|-------------|---------|---------|
| COUNT() | Count rows/values | `COUNT(*)` | Total rows |
| AVG() | Average of values | `AVG(salary)` | Mean value |
| SUM() | Sum of values | `SUM(amount)` | Total |
| MAX() | Maximum value | `MAX(price)` | Highest value |
| MIN() | Minimum value | `MIN(date)` | Earliest date |

```sql
SELECT 
    department,
    COUNT(*) as employee_count,
    ROUND(AVG(salary), 2) as avg_salary,
    MAX(salary) as highest_salary,
    MIN(salary) as lowest_salary,
    SUM(salary) as total_payroll
FROM employees
GROUP BY department;
```

## Date Functions

| Function | Description | Example | Result |
|----------|-------------|---------|---------|
| CURRENT_DATE | Current date | `CURRENT_DATE` | '2024-01-12' |
| CURRENT_TIMESTAMP | Current date/time with timezone | `CURRENT_TIMESTAMP` | '2024-01-12 10:30:00+00' |
| DATE_TRUNC | Truncate date to specified unit | `DATE_TRUNC('month', date)` | First day of month |
| EXTRACT | Get date/time field | `EXTRACT(YEAR FROM date)` | 2024 |
| AGE | Calculate interval between dates | `AGE('2024-01-01', '1990-01-01')` | '34 years' |

```sql
SELECT 
    CURRENT_DATE as today,
    CURRENT_TIMESTAMP as now_with_time,
    DATE_TRUNC('month', CURRENT_DATE) as month_start,
    EXTRACT(YEAR FROM CURRENT_DATE) as current_year,
    AGE('2024-01-12', '1990-01-01') as calculated_age,
    CURRENT_DATE + INTERVAL '1 month' as next_month;
```

## JSON Functions

| Function | Description | Example | Result |
|----------|-------------|---------|---------|
| jsonb_build_object | Create JSON object | `jsonb_build_object('key', value)` | `{"key": value}` |
| jsonb_build_array | Create JSON array | `jsonb_build_array(1,2,3)` | `[1,2,3]` |
| jsonb_extract_path | Get value by path | `jsonb_extract_path(data, 'address', 'city')` | JSON value |
| jsonb_agg | Aggregate as JSON array | `jsonb_agg(column)` | Array of values |
| jsonb_set | Set JSON field value | `jsonb_set(data, '{key}', '"value"')` | Modified JSON |

```sql
SELECT 
    jsonb_build_object('name', name, 'age', age) as person_json,
    data->'address'->>'city' as city,
    jsonb_agg(jsonb_build_object('id', id, 'value', value)) as array_json
FROM users
GROUP BY department;
```

## Math Functions

| Function | Description | Example | Result |
|----------|-------------|---------|---------|
| ROUND | Round number | `ROUND(42.123, 2)` | 42.12 |
| CEIL | Round up | `CEIL(42.1)` | 43 |
| FLOOR | Round down | `FLOOR(42.9)` | 42 |
| POWER | Raise to power | `POWER(2, 3)` | 8 |
| SQRT | Square root | `SQRT(16)` | 4 |
| RANDOM | Random value | `RANDOM()` | 0 to 1 |

```sql
SELECT 
    ROUND(price, 2) as rounded_price,
    CEIL(amount) as rounded_up,
    FLOOR(amount) as rounded_down,
    ABS(balance) as absolute_value,
    POWER(base, 2) as squared,
    SQRT(value) as square_root,
    RANDOM() * 100 as random_number
FROM orders;
```

## String Functions
| Function | Description | Example | Result |
|----------|-------------|---------|---------|
| CONCAT | Join strings | `CONCAT('Hello', ' ', 'World')` | 'Hello World' |
| LOWER | Lowercase | `LOWER('Hello')` | 'hello' |
| UPPER | Uppercase | `UPPER('hello')` | 'HELLO' |
| LENGTH | String length | `LENGTH('hello')` | 5 |
| SUBSTRING | Extract substring | `SUBSTRING('hello' from 1 for 2)` | 'he' |
| REPLACE | Replace text | `REPLACE('hello', 'l', 'L')` | 'heLLo' |

```sql
SELECT 
    CONCAT(first_name, ' ', last_name) as full_name,
    LOWER(email) as normalized_email,
    UPPER(country_code) as country_code,
    LENGTH(description) as desc_length,
    SUBSTRING(phone, 1, 3) as area_code,
    REPLACE(title, 'old', 'new') as updated_title
FROM contacts;
```

## Window Functions
| Function | Description | Example | Result |
|----------|-------------|---------|---------|
| ROW_NUMBER | Sequential row numbers | `ROW_NUMBER() OVER (ORDER BY col)` | 1, 2, 3... |
| RANK | Rank with gaps | `RANK() OVER (ORDER BY col)` | 1, 1, 3... |
| LAG | Previous row value | `LAG(col) OVER (ORDER BY col)` | Previous value |
| LEAD | Next row value | `LEAD(col) OVER (ORDER BY col)` | Next value |
| FIRST_VALUE | First value in window | `FIRST_VALUE(col) OVER (...)` | First value |

```sql
SELECT 
    department,
    employee_name,
    salary,
    ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as salary_rank,
    SUM(salary) OVER (PARTITION BY department ORDER BY salary) as running_total,
    LAG(salary) OVER (PARTITION BY department ORDER BY salary) as previous_salary,
    FIRST_VALUE(salary) OVER (PARTITION BY department ORDER BY salary) as lowest_in_dept,
    PERCENT_RANK() OVER (PARTITION BY department ORDER BY salary) as salary_percentile
FROM employees;
```

## Special Utility Functions
| Function | Description | Example | Result |
|----------|-------------|---------|---------|
| COALESCE | First non-null value | `COALESCE(null, 1, 2)` | 1 |
| NULLIF | Null if equal | `NULLIF(5, 5)` | null |
| CAST | Type conversion | `CAST('100' AS INTEGER)` | 100 |
| GREATEST | Largest of list | `GREATEST(1, 2, 3)` | 3 |
| LEAST | Smallest of list | `LEAST(1, 2, 3)` | 1 |

```sql
SELECT 
    COALESCE(nullable_field, 'default') as safe_value,
    NULLIF(value, 0) as non_zero,
    CASE 
        WHEN amount > 1000 THEN 'High'
        WHEN amount > 500 THEN 'Medium'
        ELSE 'Low'
    END as amount_category,
    CAST(string_number AS INTEGER) as converted_number
FROM transactions;
```